In [ ]:
#| default_exp anatel
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:

# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))

# Anatel

> Este módulo consolida as bases da Anatel e realiza pós-processamento dos dados obtidos.

In [ ]:
#| export
from typing import List, Tuple
from functools import cached_property
import pandas as pd
from dotenv import find_dotenv, load_dotenv
from fastcore.xtras import Path
from fastcore.parallel import parallel, partialler
from fastcore.foundation import L
from pyarrow import ArrowInvalid


from extracao.datasources.sitarweb import Stel, Radcom, SQLSERVER_PARAMS
from extracao.datasources.mosaico import MONGO_URI
from extracao.datasources.srd import SRD
from extracao.datasources.telecom import Telecom
from extracao.datasources.smp import SMP
from extracao.datasources.base import Base
from extracao.datasources.aeronautica import Aero
from extracao.datasources.connectors import SQLServer
from extracao.constants import COLUNAS, SQL_VALIDA_COORD

In [ ]:
#| export
load_dotenv(find_dotenv())

True

## Base Consolidada ANATEL

In [ ]:
# |export
class Outorgadas(Base):
    """Classe auxiliar para agregar os dados originários da Anatel"""

    def __init__(
        self,
        sql_params: dict = SQLSERVER_PARAMS,
        mongo_uri: str = MONGO_URI,
        limit: int = 0,
        n_workers: int = 10,
    ):
        self.sql_params = sql_params
        self.mongo_uri = mongo_uri
        self.limit = limit
        self.n_workers = n_workers

    @property
    def columns(self):
        return COLUNAS

    @cached_property
    def df_cache(self) -> pd.DataFrame:
        try:
            df = self._read(self.stem)
        except ValueError:
            df = pd.DataFrame(columns=self.columns)
        return df

    @property
    def stem(self):
        return "anatel"

    @staticmethod
    def _update_instance(class_instance):
        class_instance.update()
        class_instance.save()
        return class_instance.df

    @cached_property
    def extraction(self) -> L:
        sources = [
            # Aero(),
            Stel(self.sql_params),
            Radcom(self.sql_params),
            SRD(self.mongo_uri),
            Telecom(self.mongo_uri, self.limit),
            SMP(self.mongo_uri, self.limit),
        ]
        return parallel(
            Outorgadas._update_instance, sources, n_workers=len(sources), progress=True
        )

    def update_cached_df(
        self,
        df: pd.DataFrame,
    ) -> pd.DataFrame:
        """Mescla ambos dataframes eliminando os excluídos (existentes somente em df_cache)"""

        if self.df_cache.empty:
            return df

        # Merge dataframes based on all columns except "Coords_Valida_IBGE"
        merged = pd.merge(
            self.df_cache,
            df,
            on=list(df.columns),
            how="outer",
            indicator=True,
            copy=False,
            validate="one_to_one",
        )

        # Exclude rows only present in df_cache
        df_cache = merged[merged["_merge"] != "left_only"]

        # inplace=True not working
        df_cache.loc[:, ["Latitude", "Longitude"]] = df_cache.loc[
            :, ["Latitude", "Longitude"]
        ].fillna(-1)

        # Drop the _merge column
        df_cache = df_cache.drop(columns="_merge")

        # Write the new dataframe to cache
        self.df_cache = df_cache

        return self.df_cache

    @staticmethod
    def intersect_coordinates_on_poligon(
        row: pd.Series,  # DataFrame row
        sql_params: dict,  # Connection parameters to pass to SQLServer
    ) -> Tuple:  # Tuple com dados do município
        """Valida os dados de coordenadas e município em `row` no polígono dos municípios em banco corporativo do IBGE"""

        mun, cod, lat, long = (
            row.Município,
            row.Código_Município,
            row.Latitude,
            row.Longitude,
        )
        is_valid = -1
        conn = SQLServer(sql_params).connect()
        crsr = conn.cursor()
        sql = SQL_VALIDA_COORD.format(long, lat, cod)
        crsr.execute(sql)
        result = crsr.fetchone()
        if result is not None:
            mun = result.NO_MUNICIPIO
            lat = result.NU_LATITUDE
            long = result.NU_LONGITUDE
            is_valid = result.COORD_VALIDA
        conn.close()
        return mun, lat, long, is_valid

    def validate_coordinates(
        self,
        df: pd.DataFrame,  # DataFrame com os dados da Anatel
    ) -> pd.DataFrame:  # DataFrame com as coordenadas validadas na base do IBGE
        """Valida as coordenadas consultado a Base Corporativa do IBGE, excluindo o que já está no cache na versão anterior"""

        df_cache = self.update_cached_df(df)

        municipios = pd.read_csv(
            Path(__file__).parent / "datasources" / "arquivos" / "municipios.csv",
            usecols=["codigo_ibge"],
            dtype="string",
        )

        # valida os códigos de municípios
        valid_cod_mun = df_cache.Código_Município.isin(municipios.codigo_ibge)

        df_cache.loc[~valid_cod_mun, "Coords_Valida_IBGE"] = -1

        subset = df_cache.Coords_Valida_IBGE.isna()

        if linhas := list(
            df_cache.loc[
                subset, ["Município", "Código_Município", "Latitude", "Longitude"]
            ].itertuples()
        ):
            ibge = [
                "Município_IBGE",
                "Latitude_IBGE",
                "Longitude_IBGE",
                "Coords_Valida_IBGE",
            ]

            func = partialler(
                Outorgadas.intersect_coordinates_on_poligon, sql_params=self.sql_params
            )

            df_cache.loc[subset, ibge] = parallel(
                func, linhas, threadpool=True, n_workers=self.n_workers, progress=True
            )

        df_cache.loc[df_cache.Coords_Valida_IBGE == -1, "Coords_Valida_IBGE"] = pd.NA

        return df_cache

    def _format(
        self,
        dfs: List,  # List with the individual API sources
    ) -> pd.DataFrame:  # Processed DataFrame
        df = pd.concat(dfs, ignore_index=True).sort_values(
            ["Frequência", "Latitude", "Longitude"], ignore_index=True
        )

        # inplace not working!
        df.loc[:, ["Latitude", "Longitude"]] = (
            df.loc[:, ["Latitude", "Longitude"]].astype("string").fillna("0")
        )

        return df

In [ ]:
import geopandas as gpd
import pandas as pd
from fastcore.xtras import Path

In [ ]:
files = Path.cwd().parent / 'extracao' / 'datasources' / 'arquivos'

In [ ]:
files.ls()

(#8) [Path('/mnt/c/Users/rsilva/OneDrive - ANATEL/Code/anateldb/extracao/datasources/arquivos/BR_Municipios_2022'),Path('/mnt/c/Users/rsilva/OneDrive - ANATEL/Code/anateldb/extracao/datasources/arquivos/canalizacao_smp.csv'),Path('/mnt/c/Users/rsilva/OneDrive - ANATEL/Code/anateldb/extracao/datasources/arquivos/municipios.csv'),Path('/mnt/c/Users/rsilva/OneDrive - ANATEL/Code/anateldb/extracao/datasources/arquivos/radares.csv'),Path('/mnt/c/Users/rsilva/OneDrive - ANATEL/Code/anateldb/extracao/datasources/arquivos/saida'),Path('/mnt/c/Users/rsilva/OneDrive - ANATEL/Code/anateldb/extracao/datasources/arquivos/VHF_COM.csv'),Path('/mnt/c/Users/rsilva/OneDrive - ANATEL/Code/anateldb/extracao/datasources/arquivos/VHF_NAV.csv'),Path('/mnt/c/Users/rsilva/OneDrive - ANATEL/Code/anateldb/extracao/datasources/arquivos/VOR_ILS_DME_Channel.csv')]

In [ ]:
regions = gpd.read_file(files / 'BR_Municipios_2022' / 'BR_Municipios_2022.shp')

In [ ]:
df = pd.read_parquet(files / 'saida' / 'telecom.parquet.gzip')

In [ ]:
df

,Frequência,Entidade,Fistel,Número_Estação,Município,Código_Município,UF,Latitude,Longitude,Classe,...,Cod_Tipo_Antena,Polarização,Ganho_Antena,FC_Antena,Ang_MP_Antena,Ângulo_Elevação,Azimute,Altura_Antena,Perdas_Acessorias,Log
0,173.075,TRACKER DO BRASIL LTDA,50004466071,505805553,Mairiporã,3528502,SP,-23.4030555555555,-46.63583333333333,FB,...,86,V,5.8,<NA>,<NA>,<NA>,<NA>,60.0,<NA>,<NA>
1,173.075,TRACKER DO BRASIL LTDA,50004466071,684490021,Belo Horizonte,3106200,MG,-19.793333333333333,-43.968888888888834,FB,...,86,V,9.0,<NA>,<NA>,<NA>,<NA>,20.0,<NA>,<NA>
2,173.075,TRACKER DO BRASIL LTDA,50004466071,684587017,Tubarão,4218707,SC,-28.520833333333332,-48.98527777777767,FB,...,86,V,9.0,<NA>,<NA>,<NA>,<NA>,25.0,<NA>,<NA>
3,173.075,TRACKER DO BRASIL LTDA,50004466071,684610094,Florianópolis,4205407,SC,-27.588611111111,-48.533611111111,FB,...,86,V,9.0,<NA>,<NA>,<NA>,<NA>,15.0,<NA>,<NA>
4,173.075,TRACKER DO BRASIL LTDA,50004466071,684649900,João Monlevade,3136207,MG,-19.868888888888833,-43.193333333333335,FB,...,86,V,9.0,<NA>,<NA>,<NA>,<NA>,20.0,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12755,788.0,TELEFONICA BRASIL S.A.,50419058354,1014443412,MACAE,3302403,RJ,-22.623611,-39.98972,FB,...,760,X,14.9,22.0,68.0,0.0,60.0,71.5,0.0,<NA>
12756,1865.0,TELEFONICA BRASIL S.A.,50419058354,1014443447,Vitória,3205309,ES,-21.939722,-39.785278,FB,...,760,X,16.7,25.0,64.0,0.0,206.0,61.47,0.0,<NA>
12757,788.0,TELEFONICA BRASIL S.A.,50419058354,1014443447,Vitória,3205309,ES,-21.939722,-39.785278,FB,...,760,X,14.9,22.0,68.0,0.0,206.0,61.47,0.0,<NA>
12758,1862.5,TELEFONICA BRASIL S.A.,50419058354,1014443471,Vitória,3205309,ES,-21.213889,-39.997222,FB,...,760,X,16.7,25.0,64.0,0.0,187.0,58.05,0.0,<NA>


In [ ]:
from shapely.geometry import Point

# Suppose `df` is your DataFrame with 'lat' and 'lon' columns
# Convert the DataFrame into a GeoDataFrame
gdf_points = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))

# Set the same coordinate reference system (CRS) as the regions shapefile
gdf_points.crs = regions.crs


In [ ]:
gdf_points

,Frequência,Entidade,Fistel,Número_Estação,Município,Código_Município,UF,Latitude,Longitude,Classe,...,Polarização,Ganho_Antena,FC_Antena,Ang_MP_Antena,Ângulo_Elevação,Azimute,Altura_Antena,Perdas_Acessorias,Log,geometry
0,173.075,TRACKER DO BRASIL LTDA,50004466071,505805553,Mairiporã,3528502,SP,-23.4030555555555,-46.63583333333333,FB,...,V,5.8,<NA>,<NA>,<NA>,<NA>,60.0,<NA>,<NA>,POINT (-46.63583 -23.40306)
1,173.075,TRACKER DO BRASIL LTDA,50004466071,684490021,Belo Horizonte,3106200,MG,-19.793333333333333,-43.968888888888834,FB,...,V,9.0,<NA>,<NA>,<NA>,<NA>,20.0,<NA>,<NA>,POINT (-43.96889 -19.79333)
2,173.075,TRACKER DO BRASIL LTDA,50004466071,684587017,Tubarão,4218707,SC,-28.520833333333332,-48.98527777777767,FB,...,V,9.0,<NA>,<NA>,<NA>,<NA>,25.0,<NA>,<NA>,POINT (-48.98528 -28.52083)
3,173.075,TRACKER DO BRASIL LTDA,50004466071,684610094,Florianópolis,4205407,SC,-27.588611111111,-48.533611111111,FB,...,V,9.0,<NA>,<NA>,<NA>,<NA>,15.0,<NA>,<NA>,POINT (-48.53361 -27.58861)
4,173.075,TRACKER DO BRASIL LTDA,50004466071,684649900,João Monlevade,3136207,MG,-19.868888888888833,-43.193333333333335,FB,...,V,9.0,<NA>,<NA>,<NA>,<NA>,20.0,<NA>,<NA>,POINT (-43.19333 -19.86889)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12755,788.0,TELEFONICA BRASIL S.A.,50419058354,1014443412,MACAE,3302403,RJ,-22.623611,-39.98972,FB,...,X,14.9,22.0,68.0,0.0,60.0,71.5,0.0,<NA>,POINT (-39.98972 -22.62361)
12756,1865.0,TELEFONICA BRASIL S.A.,50419058354,1014443447,Vitória,3205309,ES,-21.939722,-39.785278,FB,...,X,16.7,25.0,64.0,0.0,206.0,61.47,0.0,<NA>,POINT (-39.78528 -21.93972)
12757,788.0,TELEFONICA BRASIL S.A.,50419058354,1014443447,Vitória,3205309,ES,-21.939722,-39.785278,FB,...,X,14.9,22.0,68.0,0.0,206.0,61.47,0.0,<NA>,POINT (-39.78528 -21.93972)
12758,1862.5,TELEFONICA BRASIL S.A.,50419058354,1014443471,Vitória,3205309,ES,-21.213889,-39.997222,FB,...,X,16.7,25.0,64.0,0.0,187.0,58.05,0.0,<NA>,POINT (-39.99722 -21.21389)


In [ ]:
# Spatial join points to the regions
points_with_regions = gpd.sjoin(gdf_points, regions, how="inner", predicate="within")


In [ ]:
points_with_regions

,Frequência,Entidade,Fistel,Número_Estação,Município,Código_Município,UF,Latitude,Longitude,Classe,...,Azimute,Altura_Antena,Perdas_Acessorias,Log,geometry,index_right,CD_MUN,NM_MUN,SIGLA_UF,AREA_KM2
0,173.075,TRACKER DO BRASIL LTDA,50004466071,505805553,Mairiporã,3528502,SP,-23.4030555555555,-46.63583333333333,FB,...,<NA>,60.0,<NA>,<NA>,POINT (-46.63583 -23.40306),3587,3528502,Mairiporã,SP,320.697
1330,368.7875,VERSATIL RADIOCOMUNICACAO LTDA,50416767397,1007057677,Mairiporã,3528502,SP,-23.317097222222166,-46.5897,ML,...,<NA>,1.5,<NA>,<NA>,POINT (-46.58970 -23.31710),3587,3528502,Mairiporã,SP,320.697
1331,372.9625,VERSATIL RADIOCOMUNICACAO LTDA,50416767397,1007057677,Mairiporã,3528502,SP,-23.317097222222166,-46.5897,ML,...,<NA>,1.5,<NA>,<NA>,POINT (-46.58970 -23.31710),3587,3528502,Mairiporã,SP,320.697
1332,373.1125,VERSATIL RADIOCOMUNICACAO LTDA,50416767397,1007057677,Mairiporã,3528502,SP,-23.317097222222166,-46.5897,ML,...,<NA>,1.5,<NA>,<NA>,POINT (-46.58970 -23.31710),3587,3528502,Mairiporã,SP,320.697
1333,373.3375,VERSATIL RADIOCOMUNICACAO LTDA,50416767397,1007057677,Mairiporã,3528502,SP,-23.317097222222166,-46.5897,ML,...,<NA>,1.5,<NA>,<NA>,POINT (-46.58970 -23.31710),3587,3528502,Mairiporã,SP,320.697
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12607,8426.0,INTERJATO SERVICOS DE TELECOMUNICACOES LTDA,50419573127,1011176413,Nova Cruz,2408300,RN,-6.454325,-35.376041666666666,FX,...,275.8,13.0,<NA>,<NA>,POINT (-35.37604 -6.45432),1165,2408300,Nova Cruz,RN,277.658
12614,10.835,INTERJATO SERVICOS DE TELECOMUNICACOES LTDA,50419573127,1011208072,São Miguel do Gostoso,2412559,RN,-5.1513888888888335,-35.6645305555555,FX,...,156.7,22.0,<NA>,<NA>,POINT (-35.66453 -5.15139),1211,2412559,São Miguel do Gostoso,RN,431.444
12615,8314.0,INTERJATO SERVICOS DE TELECOMUNICACOES LTDA,50419573127,1011224205,Rio do Fogo,2408953,RN,-5.3348305555555,-35.3832,FX,...,286.7,41.0,<NA>,<NA>,POINT (-35.38320 -5.33483),1172,2408953,Rio do Fogo,RN,151.097
12694,19452.5,Sempre Telecomunicacoes Ltda,50442657307,1014329180,Brumadinho,3109006,MG,-20.113217,-44.295606,FX,...,233.9,15.0,<NA>,<NA>,POINT (-44.29561 -20.11322),2340,3109006,Brumadinho,MG,639.434
